In [1]:
import numpy as np
from source.msd_datasets import *
from source.utils import split_and_load
from transformers import BertTokenizer
from source.mb_masked_lm import MusicBertMaskedLM

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# REVIEWS DATASET
reviews_dataset = MSDDatasetReviewsEchonest(tokenizer = tokenizer, length=100000)
# sMSDDatasetReviewsMFCC(tokenizer = tokenizer, length=100000)
train_dataloader, val_dataloader = split_and_load(reviews_dataset, workers=4, batch_size=8, split_size=0.99)

sound_features = 24 # MFCC
evals = 1

model = MusicBertMaskedLM(sound_features, n_convs=3).cuda()
#model.load_pretrained()

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [2]:
model.pretrain_AE(train_dataloader, epochs=1, skip_encoder=False)

In [8]:
%matplotlib widget

print(np.mean(model.AE_loss_curve[-1000:]))

curve = smooth(model.AE_loss_curve, 200)[100:-100]
plot_curve(curve, evals, color="red", metric_label="MSE")

122.2843833770752


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Model MSE: 12441.06%


In [4]:
model.pretrain_model(train_dataloader, None, epochs=1, eval_per_epoch=evals)

In [9]:
%matplotlib widget

evals = 1

cls_loss = model.cls_loss_curve
sound_loss = [x/(1024) for x in model.sound_loss_curve]
token_loss = [x/16 for x in model.tokens_loss_curve]

cls_loss = smooth(model.cls_loss_curve, 100)[100:-100]
sound_loss = smooth(sound_loss, 100)[100:-100]
token_loss = smooth(token_loss, 10)[100:-100]

#plot_curve(cls_loss, evals, color="red", metric_label="BCE")
plot_curve(token_loss, evals, color="blue", metric_label="NLL")
plot_curve(sound_loss, evals, color="green", metric_label="MSE")


plt.xlabel("Steps")
plt.ylabel("Loss")
plt.legend(["Token Loss", "Sound Loss"]) # "Cls Loss", 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Model NLL: 43.50%
Model MSE: 26.25%
